In [13]:
import pandas as pd
import numpy as np
from io import StringIO

In [21]:
df = pd.read_excel('PfamresultOvul.xlsx')

In [23]:
#for each unique 'query name', take only first row

df = df.groupby('query name').first().reset_index()

In [25]:
hits_df = df[['query name', 'accession', 'tlen', 'E-value', 'score', 'bias', 'of', 'ali from', 'ali to', 'acc', 'description of target']]

In [26]:
hits_df

,query name,accession,tlen,E-value,score,bias,of,ali from,ali to,acc,description of target
0,A0A411DYX8,PF00028.21,93,5.500000e-60,200.0,0.5,5,60,101,0.85,Cadherin domain
1,A0A411DZ00,PF00028.21,93,1.100000e-58,195.8,1.8,6,23,97,0.89,Cadherin domain
2,A0A411DZ04,PF00028.21,93,4.900000e-66,219.4,4.4,6,37,105,0.79,Cadherin domain
3,A0A411DZ06,PF00028.21,93,8.300000e-68,225.1,0.8,6,27,98,0.70,Cadherin domain
4,A0A411DZ07,PF00028.21,93,2.800000e-66,220.2,2.5,6,51,95,0.80,Cadherin domain
...,...,...,...,...,...,...,...,...,...,...,...
548,A0A7E6FUJ8,PF00041.25,85,0.000000e+00,6786.4,434.9,148,2604,2683,0.93,Fibronectin type III domain
549,A0A7E6FUU3,PF00041.25,85,0.000000e+00,6785.3,434.9,148,2697,2776,0.93,Fibronectin type III domain
550,H2EZS9,PF00520.35,241,6.800000e-51,173.0,12.2,1,165,408,0.90,Ion transport protein
551,Q5W9T5,PF00001.25,259,9.500000e-37,127.1,19.1,1,51,347,0.88,7 transmembrane receptor (rhodopsin family)


In [34]:
# Read FASTA file
with open('../data/blastresults/CLEANsmlf_Ovulgaris.fasta', 'r') as f:
    lines = f.readlines()

# Parse headers, sequences, and editing sites
ids = []
editing_sites = []
sequences = []

for i, line in enumerate(lines):
    line = line.strip()
    if i % 2 == 0:  # This is a header line
        # Split on space to separate ID and editing sites
        parts = line.replace('>', '').split(' ')
        ids.append(parts[0])
        if len(parts) > 1:
            editing_sites.append(parts[1])
        else:
            editing_sites.append(None)
    else:  # This is a sequence line
        sequences.append(line)

sequence = pd.DataFrame({
    'query name': ids,
    'Editing Sites': editing_sites,
    'Sequence': sequences
})

In [39]:
# Merge dataframes by query name (add sequence to hits_df)

seq_df = pd.merge(hits_df, sequence, on='query name', how='left')

In [40]:
def filter_sites(row):
    valid_sites = []
    sites = str(row['Editing Sites']).split(',')
    
    for site in sites:
        # Extract the number between the two characters (e.g., N161D -> 161)
        num = int(''.join(filter(str.isdigit, site)))
        
        # Check if the number falls within the range provided by "from" and "to" columns
        if row['ali from'] <= num <= row['ali to']:
            valid_sites.append(site)
            
    return ','.join(valid_sites)

# Assuming df is the dataframe from the previous script
seq_df['Editing Sites'] = seq_df.apply(filter_sites, axis=1)

# Remove rows where 'Editing Sites' is now empty
seq_df = seq_df[seq_df['Editing Sites'] != '']


In [41]:
#in Sequence column, trim every sequence by ali from ali to column values

seq_df['Sequence'] = seq_df.apply(lambda x: x['Sequence'][x['ali from']-1:x['ali to']], axis=1)

In [43]:
def adjust_sites(row):
    adjusted_sites = []
    sites = str(row['Editing Sites']).split(',')
    
    for site in sites:
        # Extract the number between the two characters (e.g., N161D -> 161)
        num = int(''.join(filter(str.isdigit, site)))
        
        # Adjust the number based on the "from" column value
        adjusted_num = num - row['ali from'] + 1
        
        # Replace the original number with the adjusted number in the site string
        adjusted_site = site.replace(str(num), str(adjusted_num))
        adjusted_sites.append(adjusted_site)
            
    return ','.join(adjusted_sites)


seq_df['Editing Sites in domain'] = seq_df.apply(adjust_sites, axis=1)

In [46]:
#create a column fasta format, from which to write a fasta file
seq_df['fasta_format'] = ">" + seq_df['query name'] + "\t" + seq_df['Editing Sites'] + "\t" + seq_df['Editing Sites in domain'] + "\n" + seq_df['Sequence']

with open("postPfam_Ovulgaris.fasta", "w") as f:
    for entry in seq_df['fasta_format']:
        f.write(entry + "\n")